In [ ]:
import pandas as pd
import bs4
from bs4 import BeautifulSoup
import requests
import numpy as np
import time
import glob
from time import sleep
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

In [ ]:
def scrape_on_off():
    seasonlist = []
    season = 2007
    while season < 2020:
        if ((int(str(season)[2:]))+1) < 10:
            season_id_end = '0'+(str((int(str(season)[2:]))+1))
        elif str(season)[2:] == '99':
            season_id_end = '00'
        else:    
            season_id_end = str((int(str(season)[2:]))+1)
        season_id_beginning = str(season)
        season_id = season_id_beginning + '-' + season_id_end
        seasonlist.append(season_id)
        season +=1
    era = [season for season in seasonlist]

    teamids = [teamid for teamid in range (1610612737,1610612768)]

    for season in era:
        for teamid in teamids:
            time.sleep(np.random.randint(0,2))
            try: 
                headers = {
                'Host': 'stats.nba.com',
                'Connection': 'keep-alive',
                'Accept': 'application/json, text/plain, */*',
                'x-nba-stats-token': 'true',
                'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36','Accept-Encoding': 'gzip, deflate, br','Accept-Language': 'en-US,en;q=0.9,hi;q=0.8',
                'x-nba-stats-origin': 'stats',
                'Referer': f'https://stats.nba.com/team/{teamid}/onoffcourt-traditional/?Season={season}&SeasonType=Regular%20Season&PerMode=Totals',
                'Accept-Encoding': 'gzip, deflate, br',
                'Accept-Language': 'en-US,en;q=0.9'}
            
                url = f"https://stats.nba.com/stats/teamplayeronoffdetails?DateFrom=&DateTo=&GameSegment=&LastNGames=0&LeagueID=00&Location=&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PaceAdjust=N&PerMode=Totals&Period=0&PlusMinus=N&Rank=N&Season={season}&SeasonSegment=&SeasonType=Regular+Season&TeamID={teamid}&VsConference=&VsDivision="
                session = requests.Session()
                retry = Retry(connect=3, backoff_factor=0.5)
                adapter = HTTPAdapter(max_retries=retry)
                session.mount('http://', adapter)
                session.mount('https://', adapter)
                r = session.get(url, headers=headers).json()

                df = pd.DataFrame(r['resultSets'][1]['rowSet'], columns = r['resultSets'][1]['headers'])
                df.to_csv(f"C:\\Users\gsteele\onoff\\{season}_{teamid}_on_off.csv", index=False)
                    
            except LookupError:
                continue


    path = r'C:\Users\gsteele\onoff'
    all_files = glob.glob(path + "/*.csv")

    csv_list = []

    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=0)
        csv_list.append(df)

    frame = pd.concat(csv_list, axis=0, ignore_index=True)

    frame.to_csv(f"C:\\Users\gsteele\onoff\\all_on_off.csv", index=False, encoding='utf-8-sig')
            
scrape_on_off()

